In [1]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import sklearn.ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, precision_score, matthews_corrcoef, confusion_matrix, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import statistics
import numpy as np
import subprocess
from pandarallel import pandarallel

In [2]:
df = pd.read_csv("data/4.2_hg38_paired_introns.tsv", sep="\t", usecols = ["sequence_maxentscan"])
df

,sequence_maxentscan
0,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...
1,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...
2,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...
3,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...
4,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...
...,...
527913,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...
527914,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...
527915,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...
527916,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...


In [3]:
df["5_prime_site_maxent_seq"] = df["sequence_maxentscan"].apply(lambda x: x[:9])
df

,sequence_maxentscan,5_prime_site_maxent_seq
0,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,ccaGTAAGT
1,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,gagGTGAGA
2,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,gagCACTGG
3,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,cttGTGAGT
4,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,tagGCAAGC
...,...,...
527913,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,aatGTAAGA
527914,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,cagGTAATG
527915,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,gggATTATG
527916,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,cgcCCTTGG


In [4]:
df["3_prime_site_maxent_seq"] = df["sequence_maxentscan"].apply(lambda x: x[-23:])
df

,sequence_maxentscan,5_prime_site_maxent_seq,3_prime_site_maxent_seq
0,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,ccaGTAAGT,AGGCTCCTGTCTCCCCCCAGgtg
1,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,gagGTGAGA,TAGCCTCTGTTCCCACGAAGgca
2,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,gagCACTGG,AACCAGGCATAGGGGAAAGAttg
3,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,cttGTGAGT,GTCATCCCCTTCACTCCCAGctc
4,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,tagGCAAGC,TAGCCTCTGTTCCCACGAAGgca
...,...,...,...
527913,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,aatGTAAGA,CCATATTCATTTTTTTCTGCcaa
527914,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,cagGTAATG,CTTTTTTTTTTTTTTTTAAGatg
527915,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,gggATTATG,TGAAAACTTCTGTGTTTCAGaaa
527916,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,cgcCCTTGG,GTCTACTTTCCAAATTACAGtct


In [7]:
df=df.drop(columns=["sequence_maxentscan"], axis=1)
df

,5_prime_site_maxent_seq,3_prime_site_maxent_seq
0,ccaGTAAGT,AGGCTCCTGTCTCCCCCCAGgtg
1,gagGTGAGA,TAGCCTCTGTTCCCACGAAGgca
2,gagCACTGG,AACCAGGCATAGGGGAAAGAttg
3,cttGTGAGT,GTCATCCCCTTCACTCCCAGctc
4,tagGCAAGC,TAGCCTCTGTTCCCACGAAGgca
...,...,...
527913,aatGTAAGA,CCATATTCATTTTTTTCTGCcaa
527914,cagGTAATG,CTTTTTTTTTTTTTTTTAAGatg
527915,gggATTATG,TGAAAACTTCTGTGTTTCAGaaa
527916,cgcCCTTGG,GTCTACTTTCCAAATTACAGtct


In [8]:
pandarallel.initialize(nb_workers=12)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [14]:
def maxentscan_5(row):
    """
    We call the tabix command, which look at the bed file to see if the row (which are entries in the intron that we extracted) overlaps with the repeat
    regions detailed in the repeat_features.bed.gz, apart from the splice-site ({row.end-2}), we also look at the small-region that proceeds (2 nt in the exon; {row.end+2}) the splice-site.
    """
    # score = !perl MaxEntScan/score5.pl {row.sequence[:9]} | cut -f2
    # # score = score[-1:1]
    # return score
    target_sequence = row
    if len(target_sequence) < 9:
	    return 0
    result = subprocess.run(f"perl MaxEntScan/score5.pl {target_sequence}", shell=True, stdout=subprocess.PIPE, text=True)
    try:
    	score = result.stdout.strip().split("\t")[1]
    except:
        print(result)
        return result
    return score

In [15]:
%%time
df["5_MaxEntScan"] = df["5_prime_site_maxent_seq"].parallel_apply(maxentscan_5)

CompletedProcess(args='perl MaxEntScan/score5.pl gccNNNNNN', returncode=0, stdout='Input string contains invalid characters. Exiting.\n')
CompletedProcess(args='perl MaxEntScan/score5.pl ctgGCCNNN', returncode=0, stdout='Input string contains invalid characters. Exiting.\n')
CompletedProcess(args='perl MaxEntScan/score5.pl ctgGCCNNN', returncode=0, stdout='Input string contains invalid characters. Exiting.\n')
CompletedProcess(args='perl MaxEntScan/score5.pl ctgGCCNNN', returncode=0, stdout='Input string contains invalid characters. Exiting.\n')
CompletedProcess(args='perl MaxEntScan/score5.pl ctgGCCNNN', returncode=0, stdout='Input string contains invalid characters. Exiting.\n')
CompletedProcess(args='perl MaxEntScan/score5.pl cnnNNNNNN', returncode=0, stdout='Input string contains invalid characters. Exiting.\n')
CompletedProcess(args='perl MaxEntScan/score5.pl nnnNNNNNN', returncode=0, stdout='Input string contains invalid characters. Exiting.\n')
CompletedProcess(args='perl MaxEnt

In [39]:
df.loc[df["5_prime_site_maxent_seq"].str.contains("N"), "5_MaxEntScan"] = 0
df[df["5_prime_site_maxent_seq"].str.contains("N")]

,5_prime_site_maxent_seq,3_prime_site_maxent_seq,5_MaxEntScan
478902,nnnNNNNNN,GCTCTTCACCTCTGACTCAGagc,0
478904,nnnNNNNNN,TTTTATATCCCACTCTGGAGctc,0
478905,nnnNNNNNN,GCTCTTCACCTCTGACTCAGagc,0
486373,gccNNNNNN,NNNNNNNNNNNGGCCAATAGaaa,0
486385,ctgGCCNNN,NNNNNNNNNNNNNNNNNNNGgcc,0
486387,ctgGCCNNN,NNNNNNNNNNNNNNNNNNNNggc,0
486390,ctgGCCNNN,NNNNNNNNNNNGGCCAATAGaaa,0
486394,ctgGCCNNN,NNNNNNNNNNNNNNNNNGGCcaa,0
486396,cnnNNNNNN,NNNNNNNNNNNGGCCAATAGaaa,0
499248,nnnNNNNNN,TTTTATATCCCACTCTGGAGctc,0


# 3-prime end

In [45]:
def maxentscan_3(row):
    """
    We call the tabix command, which look at the bed file to see if the row (which are entries in the intron that we extracted) overlaps with the repeat
    regions detailed in the repeat_features.bed.gz, apart from the splice-site ({row.end-2}), we also look at the small-region that proceeds (2 nt in the exon; {row.end+2}) the splice-site.
    """
    target_sequence = row
    if len(target_sequence) < 23:
	    return 0
    result = subprocess.run(f"perl MaxEntScan/score3.pl {target_sequence}", shell=True, stdout=subprocess.PIPE, text=True)
    try:
    	score = result.stdout.strip().split("\t")[1]
    except:
        return 0
    return score

In [44]:
df[df["3_prime_site_maxent_seq"].str.len() < 23]

,5_prime_site_maxent_seq,3_prime_site_maxent_seq,5_MaxEntScan
31,gaaCTGAct,gaaCTGActa,-7.85
9968,cggGccc,cggGccc,0
10255,gctAAGCTA,gctAAGCTAATGCTaat,-30.86
10257,cggCAGTGA,cggCAGTGATGGCAtgt,-13.67
11250,cacCTGAGg,cacCTGAGgtc,-2.49
...,...,...,...
392156,gggCCTGAC,gggCCTGACTCCCTcac,-26.47
395121,tcgCCcaa,tcgCCcaa,0
396609,cgaCTtcc,cgaCTtcc,0
397719,ctaATttt,ctaATttt,0


In [46]:
%%time
df["3_MaxEntScan"] = df["3_prime_site_maxent_seq"].parallel_apply(maxentscan_3)

CPU times: user 146 ms, sys: 463 ms, total: 609 ms
Wall time: 29min 15s


In [48]:
df

,5_prime_site_maxent_seq,3_prime_site_maxent_seq,5_MaxEntScan,3_MaxEntScan
0,ccaGTAAGT,AGGCTCCTGTCTCCCCCCAGgtg,9.09,11.90
1,gagGTGAGA,TAGCCTCTGTTCCCACGAAGgca,7.66,3.89
2,gagCACTGG,AACCAGGCATAGGGGAAAGAttg,-13.87,-31.99
3,cttGTGAGT,GTCATCCCCTTCACTCCCAGctc,7.15,7.61
4,tagGCAAGC,TAGCCTCTGTTCCCACGAAGgca,1.14,3.89
...,...,...,...,...
527913,aatGTAAGA,CCATATTCATTTTTTTCTGCcaa,5.55,-9.69
527914,cagGTAATG,CTTTTTTTTTTTTTTTTAAGatg,9.43,10.01
527915,gggATTATG,TGAAAACTTCTGTGTTTCAGaaa,-14.46,7.85
527916,cgcCCTTGG,GTCTACTTTCCAAATTACAGtct,-27.63,4.64


In [49]:
df.to_csv("data/4.3_maxentscan", sep="\t", index=False)